# Preprocessing des Données Kaggle

Dans le jeu de données Kaggle, l'objectif initial du test est de prédire une semaine, en s'appuyant sur des données couvrant la période de 2013 à 2017. 

Cependant, pour augmenter la portée de notre modèle, nous allons élargir l'ensemble de test afin de tenter de prédire **deux ans**.


In [1]:
import os
import pandas as pd

### 📥 Importation et Prétraitement des Données

In [2]:
# --Chemins--
raw_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'raw')
processed_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'processed')
training_path = os.path.join(raw_data_path, 'train.csv')
testing_path = os.path.join(raw_data_path, 'test.csv')
stores_path = os.path.join(raw_data_path, 'stores.csv')
oil_path = os.path.join(raw_data_path, 'oil.csv')
holiday_event_path = os.path.join(raw_data_path, 'holidays_events.csv')

# --Chargement des tables--
train_df = pd.read_csv(training_path)
train_df['date'] = pd.to_datetime(train_df['date'], format='%Y-%m-%d')

test_df = pd.read_csv(testing_path)
test_df['date'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d')

stores_df = pd.read_csv(stores_path)
oil_df = pd.read_csv(oil_path)
oil_df['date'] = pd.to_datetime(oil_df['date'], format='%Y-%m-%d')

holiday_event_df = pd.read_csv(holiday_event_path)
holiday_event_df['date'] = pd.to_datetime(holiday_event_df['date'], format='%Y-%m-%d')
holiday_event_df = holiday_event_df.rename(columns={'type': 'holiday_type'})

# --Fusion des données--
#
full_df = pd.concat([train_df, test_df], ignore_index=True)

#  magasins
full_df = full_df.merge(stores_df, on='store_nbr', how='left')

# pétrole
full_df = full_df.merge(oil_df, on='date', how='left')

# holiday
full_df = full_df.merge(holiday_event_df, on='date', how='left')


In [3]:
full_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,holiday_type,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,National,Ecuador,Primer dia del ano,False


### 🔍Faisons quelques vérifications sur les données

In [4]:

# --Vérification des doublons de dates--
duplicate_dates = full_df['date'].duplicated().sum()
print(f"Nombre de dates qui se répètent : {duplicate_dates}")

# --Vérification des valeurs nulles--
missing_values = full_df.isnull().sum()
print(f"Valeurs manquantes :\n{missing_values}")

# --Vérifier si chaque jour apparait--
date_range = pd.date_range(start=full_df['date'].min(), end=full_df['date'].max())
missing_days = set(date_range) - set(full_df['date'])
print(f"Jours manquants : {len(missing_days)}")
if missing_days:
    print(f"Liste des jours manquants : {sorted(missing_days)}")
else:
    print("Tous les jours sont présents dans la plage de dates.")

Nombre de dates qui se répètent : 3081160
Valeurs manquantes :
id                    0
date                  0
store_nbr             0
family                0
sales             28512
onpromotion           0
city                  0
state                 0
type                  0
cluster               0
dcoilwtico       962280
holiday_type    2578554
locale          2578554
locale_name     2578554
description     2578554
transferred     2578554
dtype: int64
Jours manquants : 4
Liste des jours manquants : [Timestamp('2013-12-25 00:00:00', freq='D'), Timestamp('2014-12-25 00:00:00', freq='D'), Timestamp('2015-12-25 00:00:00', freq='D'), Timestamp('2016-12-25 00:00:00', freq='D')]


> Chaque année il nous manque Noel, et nous avons des valeurs `null` dans la colonne `sales`

In [5]:
full_df['date'] = pd.to_datetime(full_df['date'])
# Remplir les valeurs manquantes de la colonne 'dcoilwtico' par interpolation
full_df['oil_price'] = full_df['dcoilwtico'].interpolate(limit_direction='both')

full_df['holiday'] = full_df['holiday_type'].notnull().astype(int)

full_df['month'] = full_df['date'].dt.month
full_df['year'] = full_df['date'].dt.year

# Remplir les valeurs NaN 
full_df['sales'] = full_df['sales'].fillna(0)
full_df['holiday'] = full_df['holiday'].fillna(1)
full_df['onpromotion'] = full_df['onpromotion'].fillna(0)


# Vérifier s'il reste des valeurs NaN
missing_values = full_df.isna().sum()

# Afficher les colonnes avec des valeurs manquantes
print("Valeurs manquantes après traitement :")
print(missing_values)

Valeurs manquantes après traitement :
id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type                  0
cluster               0
dcoilwtico       962280
holiday_type    2578554
locale          2578554
locale_name     2578554
description     2578554
transferred     2578554
oil_price             0
holiday               0
month                 0
year                  0
dtype: int64


### ✂️ Séparation des données pour le test : **deux ans** de data

In [6]:
# --Diviser les données en ensembles d'entraînement et de test--
training_data = full_df[full_df['date'] <= '2015-12-31']
testing_data = full_df[full_df['date'] >= '2016-01-01']

training_file_path = os.path.join(processed_data_path, 'training.csv')
testing_file_path = os.path.join(processed_data_path, 'testing.csv')

training_data.to_csv(training_file_path, index=False)
testing_data.to_csv(testing_file_path, index=False)